<a href="https://colab.research.google.com/github/NicollasAndreasmb/combinacoesCota/blob/main/Auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


---

# 📘 **Manual de Uso — Ferramenta de Combinação de Cotas (Veículo)**

---

## 🔎 **O que é esta ferramenta?**

Essa é uma ferramenta automatizada feita para ajudar vendedores a encontrarem **combinações de cotas de consórcio de veículo** que encaixem exatamente no bolso do cliente.

Ela encontra para você, em segundos, as melhores opções combinadas de crédito, entrada e parcela — tudo com base no estoque disponível em uma planilha online. E o melhor: você **não precisa tocar no código**!

---

## 🎯 **Finalidade:**

* Criar propostas personalizadas com base no valor desejado pelo cliente.
* Combinar mais de uma cota para alcançar um crédito maior.
* Ajustar entrada e parcela conforme a realidade financeira de quem está comprando.
* Economizar tempo e acelerar o processo de venda.

---

## 🛠️ **Passo a passo para utilizar:**

### 1️⃣ **Abrir o sistema:**

* A ferramenta está pronta no **Google Colab**.
* O link será enviado a você pela gestão ou ficará salvo em um local padrão da equipe.
* O sistema se conecta automaticamente ao **estoque de cotas** no Google Drive.

### 2️⃣ **Preencher os dados solicitados:**

Preencha os **quatro campos obrigatórios** logo no início:

* **Crédito:**
  Valor total que o cliente precisa de crédito para comprar o veículo.
  *Exemplo:* 50.000

* **Entrada:**
  Quanto o cliente pode dar de entrada (já com comissão da loja embutida).
  *Exemplo:* 8.000

* **Parcela (R\$):**
  Quanto o cliente está disposto a pagar por mês.
  *Exemplo:* 900

* **Tipo do Bem:**
  Escolha **“Veículo”** neste campo.

### 3️⃣ **Executar a busca:**

Clique no botão **“Buscar Combinações”**.

O sistema vai processar os dados e em poucos segundos mostrará as propostas disponíveis.

---

## 🔍 **Como funciona a busca:**

A mágica está nos bastidores! O sistema faz o seguinte:

* Filtra somente cotas de **Veículo**.
* Verifica se os valores se encaixam nas seguintes tolerâncias:

  * **Crédito:** até 10% acima ou abaixo.
  * **Entrada:** até 10% para mais ou para menos.
  * **Parcela:** varia entre **50% e 150%** do valor informado.
* Testa combinações de cotas de uma mesma administradora para montar propostas viáveis.
* Calcula automaticamente:

  * Valor total de entrada (já com comissão);
  * Crédito combinado;
  * Parcelas somadas;
  * Maior prazo entre as cotas;
  * Taxas: mensal, anual e total.

---

## 📑 **Interpretação dos resultados:**

Cada combinação encontrada será exibida nesse estilo:

```
--- Combinação 1 ---
Administradora: [Nome da administradora]
Categoria: Veículo
Códigos: [Código das cotas usadas]
Crédito total: R$ [valor]
Entrada (com comissão): R$ [valor]
Saldo Devedor: R$ [valor]
Valor total das Parcelas: R$ [valor]
Maior Prazo: [prazo em meses]
Prazo Médio: [média dos prazos]
Crédito Líquido (Real): R$ [valor líquido]
Taxa total: R$ [valor]
Taxa (%): X%
Taxa Mensal (%): X%
Taxa Anual (%): X%
```

---

## ✅ **Informações principais para apresentar ao cliente:**

* **Crédito total:** é o valor que ele vai receber.
* **Entrada:** quanto ele precisa pagar na contratação.
* **Parcelas:** valor mensal das somas.
* **Códigos:** são as cotas específicas que serão usadas.
* **Prazo:** tempo de financiamento (em meses).

---

## ✅ **Informações importantes para o vendedor:**

* Avalie se a **taxa está competitiva**.
* Veja se os valores realmente encaixam na condição passada pelo cliente.
* Use essas propostas como ponto de partida para negociar.

---

## 💡 **Recomendações importantes:**

* Se **nenhuma combinação aparecer**, tente:

  * Reduzir o valor da entrada.
  * Aumentar a parcela.
  * Verificar se o tipo está correto (deve ser **“Veículo”**).
* Lembre que a parcela aceita uma variação grande — de 50% até 150% do valor informado.

---

## 🚫 **Limitações da ferramenta:**

* Funciona somente com as cotas **já cadastradas no estoque**.
* Não reserva cotas automaticamente — **confirme no sistema oficial antes de fechar a venda**.
* Não atualiza o estoque em tempo real — ele precisa estar atualizado previamente pela equipe.

---

## 🏁 **Resumo prático:**

Quando o cliente disser:

> “Quero comprar um carro com crédito de \[X], tenho \[Y] de entrada e posso pagar até \[Z] por mês.”

Você:

* Abre o Google Colab.
* Preenche os dados com essas informações.
* Clica em **Buscar Combinações**.
* Lê a proposta e mostra para o cliente.
* Confere a disponibilidade no sistema oficial e segue com a venda.

---


In [ ]:
# @title Sistema de Busca de Combinações para Consórcio de Automóveis


import pandas as pd
import requests
import io
import itertools
from collections import defaultdict
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

# 🔧 CONFIGURAÇÕES GLOBAIS
FILE_ID = "1wYWWhbS1RFu5pglkWZXpwpRhIiS8peH9"
URL = f"https://drive.google.com/uc?export=download&id={FILE_ID}"

# Variável global para armazenar o DataFrame
df = None

def carregar_planilha():
    """Carrega e prepara a planilha para análise de consórcios de automóveis."""
    global df
    try:
        response = requests.get(URL)
        response.raise_for_status()
        data = io.BytesIO(response.content)
        df = pd.read_excel(data)

        # Padroniza nomes das colunas
        df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

        # Converte colunas numéricas
        numeric_cols = ['valor_credito', 'valor_entrada', 'valor_parcela', 'saldo_devedor', 'qtd_parcelas']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

        # Verifica colunas essenciais
        if 'tipo_do_bem' not in df.columns or 'administradora' not in df.columns:
            raise ValueError("Colunas essenciais não encontradas na planilha")

        # Remove duplicatas
        df.drop_duplicates(inplace=True)
        return True

    except Exception as e:
        print(f"Erro ao carregar planilha: {str(e)}")
        return False

def agrupar_por_administradora(dataframe):
    """Agrupa cotas por administradora."""
    grupos = defaultdict(list)
    for _, row in dataframe.iterrows():
        grupos[row['administradora']].append(row.to_dict())
    return grupos

def buscar_combinacoes(credito_desejado, entrada_desejada, parcela_desejada, administradoras=None):
    """Busca combinações de cotas para automóveis dentro dos parâmetros especificados."""
    if df is None:
        return "Planilha não carregada. Por favor, carregue a planilha primeiro."

    # Filtra por tipo de bem (Auto) e administradoras selecionadas
    df_filtrado = df[df['tipo_do_bem'].str.lower() == 'auto']
    if administradoras:
        df_filtrado = df_filtrado[df_filtrado['administradora'].isin(administradoras)]

    if df_filtrado.empty:
        return "Nenhuma cota de automóvel encontrada com os filtros aplicados"

    # Define faixas de tolerância
    credito_min, credito_max = credito_desejado * 0.93, credito_desejado * 1.07
    entrada_min, entrada_max = entrada_desejada * 0.93, entrada_desejada * 1.07
    parcela_min, parcela_max = parcela_desejada * 0.5, parcela_desejada * 1.5

    grupos = agrupar_por_administradora(df_filtrado)
    resultados = []

    for adm, cotas in grupos.items():
        for r in range(1, min(3, len(cotas)) + 1):
            for combinacao in itertools.combinations(cotas, r):
                total_credito = sum(c['valor_credito'] for c in combinacao)
                total_entrada = sum(c['valor_entrada'] for c in combinacao)
                total_parcelas = sum(c['valor_parcela'] for c in combinacao)

                if (credito_min <= total_credito <= credito_max and
                    entrada_min <= total_entrada <= entrada_max and
                    parcela_min <= total_parcelas <= parcela_max):

                    total_saldo = sum(c['saldo_devedor'] for c in combinacao)
                    valor_final = round(total_entrada + total_saldo, 2)
                    diferenca = abs(valor_final - total_credito)

                    resultados.append((adm, combinacao, diferenca))

    if not resultados:
        return "Nenhuma combinação adequada encontrada para automóveis"

    resultados.sort(key=lambda x: x[2])
    return formatar_resultados(resultados)

def formatar_resultados(resultados):
    """Formata os resultados para exibição ao usuário."""
    resposta = ""
    for idx, (adm, combinacao, _) in enumerate(resultados[:10], 1):
        codigos = [str(c['codigo']) for c in combinacao]
        total_credito = sum(c['valor_credito'] for c in combinacao)
        total_entrada = sum(c['valor_entrada'] for c in combinacao)
        total_saldo = sum(c['saldo_devedor'] for c in combinacao)
        total_parcelas = sum(c['valor_parcela'] for c in combinacao)
        maior_prazo = max(int(c['qtd_parcelas']) for c in combinacao)

        valor_final = round(total_entrada + total_saldo, 2)
        credito_real = round(total_credito - total_entrada, 2)
        taxa_total = total_saldo - credito_real
        taxa_percentual = (taxa_total / credito_real) * 100 if credito_real > 0 else 0
        taxa_mensal = taxa_percentual / maior_prazo if maior_prazo > 0 else 0

        resposta += f"""
=== Combinação {idx} ===
Administradora: {adm}
Códigos: {', '.join(codigos)}
Crédito Total: R$ {total_credito:,.2f}
Entrada: R$ {total_entrada:,.2f}
Saldo Devedor: R$ {total_saldo:,.2f}
Parcela: R$ {total_parcelas:,.2f}
Prazo: {maior_prazo} meses
Valor Final: R$ {valor_final:,.2f}
Crédito Líquido: R$ {credito_real:,.2f}
Taxa Total: {taxa_percentual:.2f}% (R$ {taxa_total:,.2f})
Taxa Mensal: {taxa_mensal:.2f}%
Taxa Anual: {taxa_mensal * 12:.2f}%
"""
    return resposta

# 👨💻 INTERFACE DO USUÁRIO
# Widgets de entrada
entrada_credito = widgets.FloatText(
    value=30000,
    description='Crédito Desejado (R$):',
    layout=widgets.Layout(width='300px')
)

entrada_entrada = widgets.FloatText(
    value=3000,
    description='Entrada Desejada (R$):',
    layout=widgets.Layout(width='300px')
)

entrada_parcela = widgets.FloatText(
    value=500,
    description='Parcela Desejada (R$):',
    layout=widgets.Layout(width='300px')
)

# Seletor de administradoras
administradoras_widget = widgets.SelectMultiple(
    description='Filtrar por administradora:',
    options=[],
    layout=widgets.Layout(width='400px', height='120px')
)

# Botão de ação
botao_busca = widgets.Button(
    description='Buscar Combinações',
    button_style='success',
    layout=widgets.Layout(width='300px')
)

# Área de resultados
saida_resultados = widgets.Output(
    layout={'border': '1px solid #eee', 'padding': '10px', 'min_height': '300px'}
)

def atualizar_administradoras():
    """Atualiza a lista de administradoras disponíveis para automóveis."""
    if df is not None:
        adm_list = df[df['tipo_do_bem'].str.lower() == 'auto']['administradora'].unique()
        administradoras_widget.options = sorted(adm_list)

def ao_clicar_botao(b):
    """Executa a busca quando o botão é clicado."""
    with saida_resultados:
        clear_output()
        try:
            resultado = buscar_combinacoes(
                entrada_credito.value,
                entrada_entrada.value,
                entrada_parcela.value,
                administradoras_widget.value
            )
            print(resultado)
        except Exception as e:
            print(f"Ocorreu um erro: {str(e)}")

# Configura eventos
botao_busca.on_click(ao_clicar_botao)

# Layout da interface
interface = widgets.VBox([
    widgets.HTML("<h2>Busca de Combinações para Consórcio de Automóveis</h2>"),
    widgets.HBox([
        widgets.VBox([entrada_credito, entrada_entrada, entrada_parcela]),
        administradoras_widget
    ]),
    botao_busca,
    saida_resultados
])

# Inicialização
if carregar_planilha():
    atualizar_administradoras()
else:
    print("Não foi possível carregar os dados. Verifique a conexão e o ID do arquivo.")

# Exibe a interface
display(interface)

⏳ Carregando planilha...
✅ Planilha carregada com sucesso
